In [22]:
# 2022-09-12 9:35 понед. Фр.
# делаю только для Телеграм план факт по добычи угля в простом тестовом сообщении план-факт

# 2022-09-13 for GiTHub

In [23]:
import os

os.chdir('.......\\reports')
os.getcwd()

'Z:\\AEON-2022\\sz\\reports'

In [24]:
import numpy as np
import pandas as pd
import datetime
import datetime as dt

from datetime import datetime, date, timedelta
import xlsxwriter
import openpyxl
import re

from dbfread import DBF


# библиотека для работы с датами и днями
from pandas.tseries.offsets import MonthEnd

# для отправки

import requests

In [25]:
pd.set_option('display.float_format', '{:.2f}'.format)

In [26]:
# read dbf

dbf = DBF('......\\tgm0908.dbf')

In [27]:
# конвертация в pandas df
df = pd.DataFrame(dbf)

In [28]:
# df.tail()

In [29]:
# последний день текущего месяца
today = dt.datetime.today().strftime("%Y-%m-%d")

dem = pd.to_datetime(today)+MonthEnd(1)

print('последний день текущего месяца: ', dem)

последний день текущего месяца:  2022-09-30 00:00:00


In [30]:
# сегодня
today2 = pd.to_datetime('today')

#Сегодня
# now = datetime.today()

# текущий день
td = today2.day

print('Текущий день ', td)

# текущий месяц
tm = today2.month

print('Текущий месяц ', tm)

# текущий год
ty = today2.year

print('Текущий год ', ty)

Текущий день  12
Текущий месяц  9
Текущий год  2022


In [31]:
# текущий месяц текстом

tm_txt = (pd.to_datetime(today2)+MonthEnd(1)).month_name()

print('Текущий месяц', tm_txt)


Текущий месяц September


In [32]:
# Поле ДАТА в формат дата

df['DATA'] = df['DATA'].apply(pd.to_datetime).copy()

df['DATA'][:5]

0   2021-10-08
1   2021-10-29
2   2021-10-29
3   2021-10-29
4   2021-10-29
Name: DATA, dtype: datetime64[ns]

In [33]:
df['Год']=pd.DatetimeIndex(df['DATA']).year
df['Месяц'] = pd.DatetimeIndex(df['DATA']).month
df['День'] = pd.DatetimeIndex(df['DATA']).day

#только год и месяц
df['YM'] = df['DATA'].dt.to_period('M')

In [34]:
# Отбор с мая 2022 год по конец текущего месяца:

df = df[(df['DATA']>= '2022-05-01') & (df['DATA']<= dem)]


# база только за 2022 год - пока отключил, работаю только по текущий месяц

# df = df[df['Год']==2022] 

In [35]:
df['SK'].unique()

array(['10.10', '51', '76.14', '60.1', '71.1', '76.9', '62.1', '60.2',
       '10.5', '10.3.1', '10.9', '68.1.1', '68.2', '68.4.1', '69.1',
       '69.2.7', '69.3.1', '69.11', '70', '76.10', '73.3', '76.41',
       '90.1.1', '19.3', '02.1', '97.21', '25', '20.1', '26', '90.9',
       '91.9', '68.4.2', '99.2.2', '10.6', '76.АВ', '19.1', '19.4',
       '99.1.1', '09', '57.2', '91.1', '52', '60.22', '08.3', '57.22',
       '60.21', '62.2'], dtype=object)

In [36]:
# df только по счету 90.01 Продажи

df = df[df['SK'].str[:6]=='90.1.1']

In [37]:
df['SKK3'].unique()

array(['Услуги техники с экипажем', 'Вскрышные работы', 'Добыча угля',
       'Образование накопителя на территории МУТ'], dtype=object)

In [38]:
# df только по номенклатуре "Добыча угля"
# 2022-09-09 добавил и другие показатели в отдельные df

df = df[df['SKK3']=='Добыча угля']

# # Вскрышные работы
# df_over = df[df['SKK3']=='Вскрышные работы']

In [39]:
# столбец в тыс.руб.
df['ST'] = (df['SUMMA']/1000).round(2)

# столбец Количество в тыс.ед.
df['KOL_T'] = (df['KOL']/1000).round(2)



In [40]:
# KOL уголь

pt1 = pd.pivot_table(df, index=['SKK1', 'SKK2', 'SKK3'], values=['KOL_T'], columns=['Год', 'Месяц'], aggfunc=np.sum, fill_value='', margins=True)

pt1

KOL_T                  
Год                                2022               All
Месяц                                 5     7     8      
SKK1             SKK2 SKK3                               
02 Горные работы 20%  Добыча угля 10.02 15.61 42.03 67.66
All                               10.02 15.61 42.03 67.66

In [42]:
pt2 = pd.pivot_table(df, index=['SKD1', 'SKK1', 'SKK3'], values=['ST'], columns=['Год', 'Месяц'], aggfunc=np.sum, fill_value='', margins=True)
pt2

ST                  \
Год                                                 2022                   
Месяц                                                  5       7       8   
SKD1                SKK1             SKK3                                  
СЕВЕРНАЯ ЗВЕЗДА ООО 02 Горные работы Добыча угля 1635.95 2337.31 7013.80   
All                                              1635.95 2337.31 7013.80   

                                                           
Год                                                   All  
Месяц                                                      
SKD1                SKK1             SKK3                  
СЕВЕРНАЯ ЗВЕЗДА ООО 02 Горные работы Добыча угля 10987.06  
All                                              10987.06

In [43]:
# сегодня в номальный вид

today = datetime.today().strftime("%Y-%m-%d %H:%M")

today

'2022-09-12 10:54'

In [45]:
# нахождение крайней даты
lastday = df['DATA'].sort_values(ascending=True).tail(1).astype('datetime64[ns]')

#
print('lastday: ', lastday)
print('-------')

#привоэу в более менее удобный формат для выдачи в текст заголовка
lastday2 = lastday.astype('str').values

print('lastday2: ', lastday2)

lastday:  9080   2022-08-31
Name: DATA, dtype: datetime64[ns]
-------
lastday2:  ['2022-08-31']


In [46]:
#Сохранение всех сводных табл в excel файл (без форматирования)
writer = pd.ExcelWriter('Z:\\.......\\tgm_coal_natur_Telegram_1.xlsx', engine='xlsxwriter')

pt1.to_excel(writer, sheet_name='pt1', header = True)
pt2.to_excel(writer, sheet_name='pt2', header = True)


# df.to_excel(writer, sheet_name='df', header = True)


writer.save()
writer.close()

c:\Users\smole\AppData\Local\Programs\Python\Python39\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [47]:
###### Работа с бюджетными показателями

# загрузка бюджета

df_b = pd.read_excel('.......\\tgm_budget_22_v2.xlsx', sheet_name='tgm')

df_b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   DATA       58 non-null     datetime64[ns]
 1   Номенкл    58 non-null     object        
 2   Ед.изм.    58 non-null     object        
 3   Кол-во     58 non-null     float64       
 4   Тариф      58 non-null     float64       
 5   Стоимость  58 non-null     float64       
dtypes: datetime64[ns](1), float64(3), object(2)
memory usage: 2.8+ KB


In [48]:
# стролбцы с периодами
df_b['Год'] = df_b['DATA'].dt.year.astype('Int64').values
df_b['Месяц'] = df_b['DATA'].dt.month.astype('Int64').values
df_b['День'] = df_b['DATA'].dt.day.astype('Int64').values

#только год и месяц
df_b['YM'] = df_b['DATA'].dt.to_period('M')

In [49]:
# df_b.tail()

In [50]:
# Отбор с мая 2022 только по конец текущего месяца:

df_b = df_b[(df_b['DATA']>= '2022-05-01') & (df_b['DATA']<= dem)]


df_b

,DATA,Номенкл,Ед.изм.,Кол-во,Тариф,Стоимость,Год,Месяц,День,YM
0,2022-05-31,Добыча угля,тыс.т.,10.02,110.50,1107.43,2022,5,31,2022-05
1,2022-06-30,Добыча угля,тыс.т.,0.00,110.50,0.00,2022,6,30,2022-06
2,2022-07-31,Добыча угля,тыс.т.,25.50,110.50,2817.75,2022,7,31,2022-07
3,2022-08-31,Добыча угля,тыс.т.,52.50,110.50,5801.25,2022,8,31,2022-08
4,2022-09-30,Добыча угля,тыс.т.,35.60,110.50,3933.80,2022,9,30,2022-09
20,2022-05-31,Вскрышные работы,тыс м3,5.00,252.58,1262.90,2022,5,31,2022-05
21,2022-06-30,Вскрышные работы,тыс м3,81.64,265.43,21669.97,2022,6,30,2022-06
22,2022-07-31,Вскрышные работы,тыс м3,228.00,264.60,60328.80,2022,7,31,2022-07
23,2022-08-31,Вскрышные работы,тыс м3,227.10,264.60,60090.66,2022,8,31,2022-08
24,2022-09-30,Вскрышные работы,тыс м3,217.20,264.60,57471.12,2022,9,30,2022-09


In [51]:
# отбор только по Номенкл "Добыча угля"

df_b = df_b[df_b['Номенкл']=='Добыча угля']

# вскрышные работы

# df_b_over = df_b[df_b['Номенкл']=='Вскрышные работы']


In [52]:
# Тестовая сводная табл по бюджету

pt1_b = pd.pivot_table(df_b, index=['Номенкл'], values=['Кол-во'], columns=['Год', 'Месяц'], aggfunc=np.sum, fill_value='', margins=True)

pt1_b

Кол-во                              
Год           2022                           All
Месяц            5    6     7     8     9       
Номенкл                                         
Добыча угля  10.02 0.00 25.50 52.50 35.60 123.62
All          10.02 0.00 25.50 52.50 35.60 123.62

In [53]:
# сводная табл по бюджету для работы

pt2_b = pd.pivot_table(df_b, index=['YM'], values=['Кол-во'], aggfunc=np.sum, fill_value='').reset_index() # , columns=['Номенкл']

# переименование столбцов
pt2_b.rename(columns={'YM':'Период', 'Кол-во': 'План'}, inplace=True)

pt2_b

,Период,План
0,2022-05,10.02
1,2022-06,0.00
2,2022-07,25.50
3,2022-08,52.50
4,2022-09,35.60


In [54]:
# сводная табл по факту для работы

pt2_f = pd.pivot_table(df, index=['YM'], values=['KOL_T'], aggfunc=np.sum, fill_value='').reset_index() # , columns=['SKK3']

# переименование столбцов

pt2_f.rename(columns={'YM':'Период', 'KOL_T': 'Факт'}, inplace=True)

pt2_f


,Период,Факт
0,2022-05,10.02
1,2022-07,15.61
2,2022-08,42.03


In [55]:
# слияние план-факт Добыча угля

m0 = pt2_b.merge(pt2_f, how = 'outer').set_index('Период') #

m0 = m0.fillna(0, inplace=False)

m0.round(2)

m0

,План,Факт
Период,,
2022-05,10.02,10.02
2022-06,0.00,0.00
2022-07,25.50,15.61
2022-08,52.50,42.03
2022-09,35.60,0.00


In [56]:
# Добыча угля
m0['Абс.откл'] = (m0['Факт']- m0['План'])
m0['Отн.откл%'] = (m0['Абс.откл'] / m0['План'] *100).round(2)
m0['Отн.откл'] = (m0['Абс.откл'] / m0['План']).round(4)

m0 = m0.fillna(0, inplace=False)

m0

,План,Факт,Абс.откл,Отн.откл%,Отн.откл
Период,,,,,
2022-05,10.02,10.02,-0.00,-0.02,-0.00
2022-06,0.00,0.00,0.00,0.00,0.00
2022-07,25.50,15.61,-9.89,-38.78,-0.39
2022-08,52.50,42.03,-10.47,-19.94,-0.20
2022-09,35.60,0.00,-35.60,-100.00,-1.00


In [57]:
# Добыча угля только нужные столбцы
m1 = m0[['План','Факт', 'Абс.откл', 'Отн.откл%']]

m1

,План,Факт,Абс.откл,Отн.откл%
Период,,,,
2022-05,10.02,10.02,-0.00,-0.02
2022-06,0.00,0.00,0.00,0.00
2022-07,25.50,15.61,-9.89,-38.78
2022-08,52.50,42.03,-10.47,-19.94
2022-09,35.60,0.00,-35.60,-100.00


In [58]:
# взять данные из последней строки (данные за последний месяц) в переменные

План_У_тм = (m1.iloc[-1][0]).round(2)

Факт_У_тм = (m1.iloc[-1][1]).round(2)

абс_У_тм = (Факт_У_тм - План_У_тм).round(2)
отн_У_тм = ((абс_У_тм/План_У_тм)*100).round(2)

print(План_У_тм)
print(Факт_У_тм)
print(абс_У_тм)
print(отн_У_тм)

35.6
0.0
-35.6
-100.0


In [59]:
# переменные с итогами
План_У = m0['План'].sum().round(2)
Факт_У = m0['Факт'].sum().round(2)
абс_У = (Факт_У - План_У).round(2)
отн_У = ((абс_У/План_У)*100).round(2)

print('План: ', План_У)
print('Факт: ', Факт_У)
print('абс: ', абс_У)
print('отн %: ', отн_У)

План:  123.62
Факт:  67.66
абс:  -55.96
отн %:  -45.27


In [60]:
####
# Передача данных в Телеграм канал


In [61]:
# Текст для отправки

text1 = f'Добрый день,\n\
Передаю для информации отчет по добычи угля на проекте "......".\n\
\n\
Всего c мая 2022 по текущий месяц ({tm_txt}-{ty}) добыча угля составила (тыс.тонн):\n\
    План:\t {План_У}\n\
    Факт:\t {Факт_У}\n\
    Абс.откл.:\t {абс_У}\n\
    Отн.откл.:\t {отн_У}%\n\n\
Всего за текущий месяц ({tm_txt}-{ty}) добыча угля составила (тыс.тонн):\n\
    План:\t {План_У_тм}\n\
    Факт:\t {Факт_У_тм}\n\
    Абс.откл.:\t {абс_У_тм}\n\
    Отн.откл.:\t {отн_У_тм}%\n\n\
Источник данных: 1С ..... (счет 90.01, номенклатура "Добыча угля").\n\
Плановые данные взяты из подготавливаемого в настоящее время бюджета.\n\n\
С уважением,\n\
Смоленышев Олег\n\
{today}'

print(text1)

Добрый день,
Передаю для информации отчет по добычи угля на проекте "Северная Звезда".

Всего c мая 2022 по текущий месяц (September-2022) добыча угля составила (тыс.тонн):
    План:	 123.62
    Факт:	 67.66
    Абс.откл.:	 -55.96
    Отн.откл.:	 -45.27%

Всего за текущий месяц (September-2022) добыча угля составила (тыс.тонн):
    План:	 35.6
    Факт:	 0.0
    Абс.откл.:	 -35.6
    Отн.откл.:	 -100.0%

Источник данных: 1С ТаймырГорМаш (счет 90.01, номенклатура "Добыча угля").
Плановые данные взяты из подготавливаемого в настоящее время бюджета.

С уважением,
Смоленышев Олег
2022-09-12 10:54


In [62]:
# Отправки сообщения в Телеграм канал Smolenishev_reports

api_token = '.................'
req1 = requests.get('https://api.telegram.org/bot{}/sendMessage'.format(api_token), params=dict(chat_id=........., text=text1))